In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import pandas_profiling as pp
import matplotlib.pyplot as plt
import seaborn as sbn
import warnings

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold, KFold, train_test_split
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error as mae
from sklearn import metrics

import time

warnings.filterwarnings('ignore')
sbn.set(style='white')

In [ ]:
import os
os.listdir('../input/')

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train_data = pd.read_csv('../input/simple-molecular-geometry-features/train_geom.csv')
test_data = pd.read_csv('../input/simple-molecular-geometry-features/test_geom.csv')
structure_data = pd.read_csv('../input/champs-scalar-coupling/structures.csv')
scalar_data = pd.read_csv('../input/champs-scalar-coupling/scalar_coupling_contributions.csv')

In [ ]:
pp.ProfileReport(train_data)

In [ ]:
pp.ProfileReport(structure_data)

In [ ]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structure_data, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

train_data = map_atom_info(train_data, 0)
train_data = map_atom_info(train_data, 1)

test_data = map_atom_info(test_data, 0)
test_data = map_atom_info(test_data, 1)

del structure_data

In [ ]:
def calculate_distance(dataframe):
    dataframe['distance'] = ((dataframe['x_0']-dataframe['x_1'])**2 +
                             (dataframe['y_0']-dataframe['y_1'])**2 +
                             (dataframe['z_0']-dataframe['z_1'])**2)**0.5
    return dataframe

train_data = calculate_distance(train_data)
test_data = calculate_distance(test_data)

In [ ]:
train_data = train_data.merge(scalar_data, how='left', on=['molecule_name', 'atom_index_0', 'atom_index_1', 'type'])

In [ ]:
del scalar_data
train_data = reduce_mem_usage(train_data)
test_data = reduce_mem_usage(test_data)

In [ ]:
def handle_type(df, verbose=True):
    df['type_number'] = [int(x[0]) for x in df['type']]
    df['type_bond'] = [x[1:] for x in df['type']]
    if verbose:
        print("Added 2 columns, Removed 1 column")
    return df

train_data = handle_type(train_data)
test_data = handle_type(test_data)

In [ ]:
def handle_encodings(df_train, df_test, columns, verbose=True):
    for col in columns:
        encoder = LabelEncoder()
        df_train[col] = encoder.fit_transform(df_train[col])
        df_test[col] = encoder.transform(df_test[col])
        if verbose:
            print("Encoded {} column".format(col))
    return df_train, df_test

columns = ['atom_0', 'atom_1', 'type_bond', 'type']
train_data, test_data = handle_encodings(train_data, test_data, columns)

In [ ]:
molecule_train = train_data.pop('molecule_name')
molecule_test = test_data.pop('molecule_name')
train_data.drop(['id'], axis=1, inplace=True)
test_data.drop(['id'], axis=1, inplace=True)

In [ ]:
train_data = reduce_mem_usage(train_data)
test_data = reduce_mem_usage(test_data)

In [ ]:
correlation = train_data.corr()
f, ax = plt.subplots(figsize=(15,15))
cmap = sbn.diverging_palette(220, 10, as_cmap=True)
sbn.heatmap(correlation, cmap=cmap, square=True)

"distance", "type number", "fc", "dso", "type" seem like very important features from this heatmap. Also "bond_angle_plane" is useless feature. Can remove it safely.

In [ ]:
sbn.jointplot(x='fc', y='scalar_coupling_constant', data=train_data, kind='hex')

Seems like there is a very strong correlation between 'fc' and are "target". Strange we do not have value for fc. Maybe prediciting value of 'fc' would get us closer to the actual values of "target"

In [ ]:
sbn.jointplot(x='dso', y='scalar_coupling_constant', data=train_data, kind='hex')

Let us try and predict values of "fc" using attributes that are highly related to it namely 'distance', 'type_number', 'type_bond', 'atom_index_1', 'atom_1', 'bond_angle_axis'. Though 'sd' and 'dso' too but that values aren't available to us in test file.

In [ ]:
fc_train = pd.concat([train_data['distance'], 
                      train_data['type_number'], 
                      train_data['type_bond'], 
                      train_data['atom_index_1'], 
                      train_data['atom_1'], 
                      train_data['bond_angle_axis'],
                      train_data['type'],
                      train_data['fc']], axis=1)
fc_test = pd.concat([test_data['distance'], 
                     test_data['type_number'], 
                     test_data['type_bond'], 
                     test_data['atom_index_1'], 
                     test_data['atom_1'], 
                     test_data['bond_angle_axis'],
                     test_data['type']], axis=1)

In [ ]:
sbn.jointplot(x='distance', y='fc', data=fc_train, kind='hex')

In [ ]:
sbn.jointplot(x='bond_angle_axis', y='fc', data=fc_train, kind='hex')

In [ ]:
sbn.distplot(fc_train['fc'])

Outlier warning?

In [ ]:
sbn.countplot(x='type_number', data=fc_train)

In [ ]:
sbn.jointplot(x='type_number', y='fc', data=fc_train, kind='hex')

In [ ]:
sbn.jointplot(x='type', y='fc', data=fc_train, kind='hex')

In [ ]:
sbn.jointplot(x='type_bond', y='fc', data=fc_train, kind='hex')

In [ ]:
def check_outlier_presence(train_df, verbose=True):
    train_df = train_df.sort_values('fc', ascending=True)
    rows_count = train_df.shape[0]
    q1_index = int(rows_count/4)
    q3_index = 3*q1_index
    q1_value = train_df.iloc[q1_index]['fc']
    q3_value = train_df.iloc[q3_index]['fc']
    iqr = q3_value-q1_value
    outlier_df = train_df.loc[train_df['fc']>((1.5*iqr)+q3_value)]
    if verbose:
        print("If {} is more than {}, it is outlier".format('fc', q3_value+1.5*iqr))
        print("Number of outliers, mathematically: {}".format(outlier_df.shape[0]))
        print("Percentage of outliers, mathematically: {}".format(100*outlier_df.shape[0]/train_df.shape[0]))
    strong_outlier_df = outlier_df.loc[outlier_df['fc']>(q3_value+3*iqr)]
    if verbose:
        print("If {} is more than {}, it is strong outlier".format('fc', q3_value+3*iqr))
        print("Number of strong outliers: {}".format(strong_outlier_df.shape[0]))
        print("Percentage of strong outliers: {}".format(100*strong_outlier_df.shape[0]/train_df.shape[0]))
    

check_outlier_presence(fc_train)

I don't believe in this numbers.. Let's verify what these outliers want to tell.

In [ ]:
plt.figure(figsize=(10,10))
sbn.scatterplot(x='fc', y='distance', hue='type', data=fc_train)
plt.show()

No outliers according to this as everything greater than 100 belongs to same class and this might be a useful information for model.

In [ ]:
fc_target = fc_train.pop('fc')

In [ ]:
params = {"learning_rate" : 0.1,
          "depth": 9,
          'metric':'MAE',
          "loss_function": "MAE"}

splits = list(KFold(n_splits=5, random_state=42).split(fc_train, fc_target))


oof = np.zeros(len(fc_train))
predictions = np.zeros(len(fc_test))
for i, (train_idx, val_idx) in enumerate(splits):
    print(f'Fold {i + 1}')
    x_train = np.array(fc_train)
    y_train = np.array(fc_target)
    trn_data = lgb.Dataset(x_train[train_idx.astype(int)], label=y_train[train_idx.astype(int)])
    val_data = lgb.Dataset(x_train[val_idx.astype(int)], label=y_train[val_idx.astype(int)])
    
    num_round = 7500
    clf = lgb.train(params, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=500, early_stopping_rounds = 350)
    oof[val_idx] = clf.predict(x_train[val_idx], num_iteration=clf.best_iteration)
    
    predictions += clf.predict(test_data, num_iteration=clf.best_iteration) / 5

In [ ]:
fc = pd.DataFrame({'fc': predictions})
test_data = test_data.join(fc)
# test_data.to_csv('test_with_fc.csv', index=True)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.drop(['bond_angle_plane', 'sd', 'pso', 'dso'], axis=1, inplace=True)
test_data.drop(['bond_angle_plane'], axis=1, inplace=True)

In [ ]:
def create_features(df):
    df['molecule_couples'] = df.groupby('molecule_name')['id'].transform('count')
    df['molecule_dist_mean'] = df.groupby('molecule_name')['distance'].transform('mean')
    df['molecule_dist_min'] = df.groupby('molecule_name')['distance'].transform('min')
    df['molecule_dist_max'] = df.groupby('molecule_name')['distance'].transform('max')
    df['atom_0_couples_count'] = df.groupby(['molecule_name', 'atom_index_0'])['id'].transform('count')
    df['atom_1_couples_count'] = df.groupby(['molecule_name', 'atom_index_1'])['id'].transform('count')
    
    df[f'molecule_atom_index_0_x_1_std'] = df.groupby(['molecule_name', 'atom_index_0'])['x_1'].transform('std')
    df[f'molecule_atom_index_0_y_1_mean'] = df.groupby(['molecule_name', 'atom_index_0'])['y_1'].transform('mean')
    df[f'molecule_atom_index_0_y_1_mean_diff'] = df[f'molecule_atom_index_0_y_1_mean'] - df['y_1']
    df[f'molecule_atom_index_0_y_1_mean_div'] = df[f'molecule_atom_index_0_y_1_mean'] / df['y_1']
    df[f'molecule_atom_index_0_y_1_max'] = df.groupby(['molecule_name', 'atom_index_0'])['y_1'].transform('max')
    df[f'molecule_atom_index_0_y_1_max_diff'] = df[f'molecule_atom_index_0_y_1_max'] - df['y_1']
    df[f'molecule_atom_index_0_y_1_std'] = df.groupby(['molecule_name', 'atom_index_0'])['y_1'].transform('std')
    df[f'molecule_atom_index_0_z_1_std'] = df.groupby(['molecule_name', 'atom_index_0'])['z_1'].transform('std')
    df[f'molecule_atom_index_0_dist_mean'] = df.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('mean')
    df[f'molecule_atom_index_0_dist_mean_diff'] = df[f'molecule_atom_index_0_dist_mean'] - df['distance']
    df[f'molecule_atom_index_0_dist_mean_div'] = df[f'molecule_atom_index_0_dist_mean'] / df['distance']
    df[f'molecule_atom_index_0_dist_max'] = df.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('max')
    df[f'molecule_atom_index_0_dist_max_diff'] = df[f'molecule_atom_index_0_dist_max'] - df['distance']
    df[f'molecule_atom_index_0_dist_max_div'] = df[f'molecule_atom_index_0_dist_max'] / df['distance']
    df[f'molecule_atom_index_0_dist_min'] = df.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('min')
    df[f'molecule_atom_index_0_dist_min_diff'] = df[f'molecule_atom_index_0_dist_min'] - df['distance']
    df[f'molecule_atom_index_0_dist_min_div'] = df[f'molecule_atom_index_0_dist_min'] / df['distance']
    df[f'molecule_atom_index_0_dist_std'] = df.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('std')
    df[f'molecule_atom_index_0_dist_std_diff'] = df[f'molecule_atom_index_0_dist_std'] - df['distance']
    df[f'molecule_atom_index_0_dist_std_div'] = df[f'molecule_atom_index_0_dist_std'] / df['distance']
    df[f'molecule_atom_index_1_dist_mean'] = df.groupby(['molecule_name', 'atom_index_1'])['distance'].transform('mean')
    df[f'molecule_atom_index_1_dist_mean_diff'] = df[f'molecule_atom_index_1_dist_mean'] - df['distance']
    df[f'molecule_atom_index_1_dist_mean_div'] = df[f'molecule_atom_index_1_dist_mean'] / df['distance']
    df[f'molecule_atom_index_1_dist_max'] = df.groupby(['molecule_name', 'atom_index_1'])['distance'].transform('max')
    df[f'molecule_atom_index_1_dist_max_diff'] = df[f'molecule_atom_index_1_dist_max'] - df['distance']
    df[f'molecule_atom_index_1_dist_max_div'] = df[f'molecule_atom_index_1_dist_max'] / df['distance']
    df[f'molecule_atom_index_1_dist_min'] = df.groupby(['molecule_name', 'atom_index_1'])['distance'].transform('min')
    df[f'molecule_atom_index_1_dist_min_diff'] = df[f'molecule_atom_index_1_dist_min'] - df['distance']
    df[f'molecule_atom_index_1_dist_min_div'] = df[f'molecule_atom_index_1_dist_min'] / df['distance']
    df[f'molecule_atom_index_1_dist_std'] = df.groupby(['molecule_name', 'atom_index_1'])['distance'].transform('std')
    df[f'molecule_atom_index_1_dist_std_diff'] = df[f'molecule_atom_index_1_dist_std'] - df['distance']
    df[f'molecule_atom_index_1_dist_std_div'] = df[f'molecule_atom_index_1_dist_std'] / df['distance']
    df[f'molecule_atom_1_dist_mean'] = df.groupby(['molecule_name', 'atom_1'])['distance'].transform('mean')
    df[f'molecule_atom_1_dist_min'] = df.groupby(['molecule_name', 'atom_1'])['distance'].transform('min')
    df[f'molecule_atom_1_dist_min_diff'] = df[f'molecule_atom_1_dist_min'] - df['distance']
    df[f'molecule_atom_1_dist_min_div'] = df[f'molecule_atom_1_dist_min'] / df['distance']
    df[f'molecule_atom_1_dist_std'] = df.groupby(['molecule_name', 'atom_1'])['distance'].transform('std')
    df[f'molecule_atom_1_dist_std_diff'] = df[f'molecule_atom_1_dist_std'] - df['distance']
    df[f'molecule_type_0_dist_std'] = df.groupby(['molecule_name', 'type_number'])['distance'].transform('std')
    df[f'molecule_type_0_dist_std_diff'] = df[f'molecule_type_0_dist_std'] - df['distance']
    df[f'molecule_type_dist_mean'] = df.groupby(['molecule_name', 'type'])['distance'].transform('mean')
    df[f'molecule_type_dist_mean_diff'] = df[f'molecule_type_dist_mean'] - df['distance']
    df[f'molecule_type_dist_mean_div'] = df[f'molecule_type_dist_mean'] / df['distance']
    df[f'molecule_type_dist_max'] = df.groupby(['molecule_name', 'type'])['distance'].transform('max')
    df[f'molecule_type_dist_min'] = df.groupby(['molecule_name', 'type'])['distance'].transform('min')
    df[f'molecule_type_dist_std'] = df.groupby(['molecule_name', 'type'])['distance'].transform('std')
    df[f'molecule_type_dist_std_diff'] = df[f'molecule_type_dist_std'] - df['distance']

    df = reduce_mem_usage(df)
    return df

train_data['molecule_name'] = molecule_train
test_data['molecule_name'] = molecule_test
train_data['id'] = list(np.arange(len(train_data)))
test_data['id'] = list(np.arange(len(test_data)))
train_data = create_features(train_data)
test_data = create_features(test_data)

In [ ]:
params = {"learning_rate" : 0.1,
          "depth": 11,
          'metric':'MAE',
          "loss_function": "MAE"}

target_data = train_data.pop('scalar_coupling_constant')
train_data.drop(['molecule_name', 'id'], axis=1, inplace=True)
test_data.drop(['molecule_name', 'id'], axis=1, inplace=True)
splits = list(KFold(n_splits=3, random_state=42).split(train_data, target_data))


oof = np.zeros(len(train_data))
predictions = np.zeros(len(test_data))
for i, (train_idx, val_idx) in enumerate(splits):
    print(f'Fold {i + 1}')
    x_train = np.array(train_data)
    y_train = np.array(target_data)
    trn_data = lgb.Dataset(x_train[train_idx.astype(int)], label=y_train[train_idx.astype(int)])
    val_data = lgb.Dataset(x_train[val_idx.astype(int)], label=y_train[val_idx.astype(int)])
    
    num_round = 7000
    clf = lgb.train(params, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=500, early_stopping_rounds = 350)
    oof[val_idx] = clf.predict(x_train[val_idx], num_iteration=clf.best_iteration)
    
    predictions += clf.predict(test_data, num_iteration=clf.best_iteration) / 3

In [ ]:
submission = pd.read_csv('../input/champs-scalar-coupling/sample_submission.csv')

submission['scalar_coupling_constant'] = predictions
submission.to_csv('submission_single_model.csv',index=False)